In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

from fastai.vision.all import *
from fastai.text.all import *
from pathlib import Path

import torch
torch.cuda.empty_cache()
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision

import torchtext
from torchtext.data import get_tokenizer   # for tokenization
from collections import Counter     # for tokenizer

import torchvision.transforms as T
import torchvision.models as models

import matplotlib.pyplot as plt
# import matplotlib.image as Image
import PIL
from PIL import Image

# for the bleu scores
from nltk.translate import bleu
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
from main import *
from model_attention import EncoderDecoder
# from model_baseline import CNNToRNN

In [17]:
# Getting CUDA
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device :-", device)

Using device :- cuda:0


In [18]:
CAPTIONS_PATH = "Flickr_8k/captions.txt"
IMAGES_PATH = "Flickr_8k/Images/"

In [19]:
transforms = T.Compose([
    T.Resize(256),
    T.CenterCrop(384),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = customDataset(
    image_dir = IMAGES_PATH,
    captions_file = CAPTIONS_PATH,
    transform = transforms
)

In [20]:
with open('vocab.pkl', 'wb') as file:
    pickle.dump(dataset.vocab, file)

In [21]:
validation_split = .1
shuffle_dataset = False
random_seed= 42

In [22]:
batch_size = 8
val_ratio = 0.1
train_loader,validator_loader = train_val_split(dataset,val_ratio,batch_size)

In [23]:
# Initializing Variable for attention model
embed_sz = 300
vocab_sz = len(dataset.vocab)
att_dim = 128
enc_dim = 2048
dec_dim = 256
learning_rate = 3e-4

model = EncoderDecoder(
    embed_sz = 300,
    vocab_sz = len(dataset.vocab),
    att_dim = 128,
    enc_dim = 2048,
    dec_dim = 256
).to(device)

model.enc_dim

2048

In [24]:
criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.sentenceToIndex["<PAD>"])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
epochs = 1
print_each = 100
train(
    epochs=epochs,
    print_each=print_each,
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    dls=train_loader,
    vocab_sz=vocab_sz,
    validation_loader=validator_loader,
    dataset=dataset,
    device=device
)

In [25]:
Model_path = './attention.pth'
model = load_model(Model_path,model)
descriptors = getDescriptors(CAPTIONS_PATH)

y_true,y_pred= make_predictions(model,validator_loader,batch_size,dataset,descriptors,device)

No of captions: 40455


In [26]:
calc_BLEU_score(y_true,y_pred)

BLEU-1: 0.673160
BLEU-2: 0.467582
BLEU-3: 0.343748
BLEU-4: 0.204839


In [27]:
calc_meteor_score(y_true,y_pred)

Meteor score:  0.5431532500432575
